In [1]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv 

--2021-11-26 19:30:36--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2021-11-26 19:30:36--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 404 Not Found
2021-11-26 19:30:36 ERROR 404: Not Found.

--2021-11-26 19:30:36--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2021-11-26 19:30:36--  https:

In [2]:
! pip install pymorphy2

In [3]:
! pip install torchmetrics

In [4]:
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter
from nltk.tokenize import WordPunctTokenizer
from pymorphy2 import MorphAnalyzer

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torchmetrics.functional import f1

tokenizer = WordPunctTokenizer()
morph = MorphAnalyzer()

In [5]:
def seed_everything(seed: int): # код из тетрадки по иаду, чтобы зафиксировать все
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [6]:
seed_everything(42)

# Часть 0.

Создаем датасет:

In [7]:
pos_tweets = pd.read_csv('positive.csv', encoding='utf-8', sep=';', header=None,  names=[0,1,2,'text','tone',5,6,7,8,9,10,11])
neg_tweets = pd.read_csv('negative.csv', encoding='utf-8', sep=';', header=None, names=[0,1,2,'text','tone',5,6,7,8,9,10,11] )
neg_tweets['tone'] = 0

In [8]:
all_tweets_data = pos_tweets.append(neg_tweets)
print(len(all_tweets_data))

226834


In [9]:
all_tweets_data.head()

,0,1,2,text,tone,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [10]:
def tokenize(text):
    #tokens = tokenizer.tokenize(text.lower())
    tokens = tokenizer.tokenize(text.lower())
    lemmas = [morph.parse(token)[0].normal_form for token in tokens if token not in punctuation]
    return ' '.join(lemmas)

In [11]:
all_tweets_data['lemmas'] = all_tweets_data['text'].apply(tokenize)

In [12]:
tweets_data = shuffle(all_tweets_data[['text','tone', 'lemmas']])

In [13]:
train_sentences, val_sentences = train_test_split(tweets_data, test_size=0.2)

Теперь создадим vocab2id для токенов, лемм и символов:

In [14]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [15]:
word_vocab = Counter()
lemma_vocab = Counter()
symbol_vocab = Counter()

for i, text in tweets_data.iterrows():
    word_vocab.update(preprocess(text[0]))
    lemma_vocab.update(text[2].split())
    symbol_vocab.update(list(text[0]))
print('всего уникальных токенов:', len(word_vocab))
print('всего уникальных лемм:', len(lemma_vocab))
print('всего уникальных символов:', len(symbol_vocab))

всего уникальных токенов: 366250
всего уникальных лемм: 224582
всего уникальных символов: 456


In [16]:
def create2id(vocab, threshold):
    filtered_vocab = set()
    for word in vocab:
        if vocab[word] > threshold:
            filtered_vocab.add(word)
    print(f'уникальных токенов, втретившихся больше {threshold} раз:', len(filtered_vocab))

    word2id = {'PAD':0}
    for word in filtered_vocab:
        word2id[word] = len(word2id)
    id2word = {i:word for word, i in word2id.items()}
    return word2id, id2word

In [17]:
word2id, id2word = create2id(word_vocab, 2)
lemma2id, id2lemma = create2id(lemma_vocab, 2)
symbol2id, id2symbol = create2id(symbol_vocab, 5)

уникальных токенов, втретившихся больше 2 раз: 64423
уникальных токенов, втретившихся больше 2 раз: 45069
уникальных токенов, втретившихся больше 5 раз: 234


In [18]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

# Часть I.


Зададим датасет:

In [19]:
class TweetsDataset(Dataset):

    def __init__(self, dataset, word2id, DEVICE, text='text'):
        self.dataset = dataset[text].values
        self.word2id = word2id
        self.length = dataset.shape[0]
        self.target = torch.Tensor(dataset['tone'].values)
        self.device = DEVICE


    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        tokens = self.preprocess(self.dataset[index]) # токенизируем
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        y = self.target[index]
        return ids, y
    
    def preprocess(self, text):
        tokens = text.lower().split()
        tokens = [token.strip(punctuation) for token in tokens]
        tokens = [token for token in tokens if token not in punctuation]
        return tokens

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, y = list(zip(*batch))
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      y = torch.Tensor(y).to(self.device)
      return padded_ids,  y

In [20]:
train_dataset = TweetsDataset(train_sentences, word2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=64)
val_dataset = TweetsDataset(val_sentences, word2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=64)

Теперь зададим архитектуру модели:

In [21]:
class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, pretrained=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        if pretrained:
          self.embedding.from_pretrained(torch.tensor(weights), freeze=True)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.conv = nn.Conv1d(in_channels=180, out_channels=100, kernel_size=2)
        self.pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=100, out_features=1)
        self.dropout = nn.Dropout(p=0.5)
        self.out = nn.Sigmoid()

    def forward(self, word):
        #batch_size x seq_len
        embedded = self.embedding(word)
        embedded = embedded.transpose(1,2)
        bigrams = self.relu(self.bigrams(embedded))
        trigrams = self.relu(self.trigrams(embedded))
        concat = torch.cat((bigrams, trigrams), 1)
        conv = self.pooling(self.conv(concat)).max(2)[0]
        logits = self.hidden(conv) 
        logits = self.out(logits)      
        return logits

In [22]:
def train(model, iterator, optimizer, criterion, metric):
    print('Training...')
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах
    epoch_metric = 0
    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (texts, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(texts).squeeze()  #прогоняем данные через модель
        loss = criterion(preds, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        
        batch_metric = metric(preds, ys.long())
        epoch_metric += batch_metric
        #if not (i + 1) % 20:
        #    print(f'Train loss: {loss.item()}, Train f1: {batch_metric}')
            #print(f'Train loss: {epoch_loss/i}, Train f1: {epoch_metric/i}')     
    print(f'F-score: {epoch_metric / len(iterator)}, loss: {epoch_loss / len(iterator)}') 
    return epoch_metric / len(iterator), epoch_loss / len(iterator) # возвращаем среднее значение по всей выборке

In [23]:
def evaluate(model, iterator, criterion, metric):
    print("\nValidating...")
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, ys) in enumerate(iterator):   
            preds = model(texts).squeeze()  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            batch_metric = metric(preds, ys.long())
            epoch_metric += batch_metric

    print(f'F-score: {epoch_metric / len(iterator)}, loss: {epoch_loss / len(iterator)}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator) # возвращаем среднее значение по всей выборке

In [24]:
def train_model(model, lr, epochs):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss() 

    model = model.to(DEVICE)
    criterion = criterion.to(DEVICE)

    losses = []
    losses_eval = []
    f1s = []
    f1s_eval = []

    for i in range(epochs):
        print(f'\nstarting Epoch {i}')
        f1_value, epoch_loss = train(model, train_iterator, optimizer, criterion, metric=f1)
        losses.append(epoch_loss)
        f1s.append(f1_value)

        f1_value_on_test, epoch_loss_on_test = evaluate(model, val_iterator, criterion, metric=f1)
        losses_eval.append(epoch_loss_on_test)
        f1s_eval.append(f1_value_on_test)

Тренируем модель с дефолтными параметрами:

In [25]:
model = CNN(len(word2id), 8)
train_model(model, 0.001, 3)


starting Epoch 0
Training...


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


F-score: 0.6242083311080933, loss: 0.6217950288369222

Validating...
F-score: 0.6830612421035767, loss: 0.5853740812945265

starting Epoch 1
Training...
F-score: 0.7102341651916504, loss: 0.545849538530382

Validating...
F-score: 0.7305583953857422, loss: 0.5490723389328283

starting Epoch 2
Training...
F-score: 0.7540391087532043, loss: 0.4919508190980583

Validating...
F-score: 0.7225967645645142, loss: 0.5226638578561532


F-score равен 0.72, поэтому есть куда улучшать!

### 1.2. Используем  эмбеддинги FastText

Попробуем натренировать FastText на нашем корпусе и загрузить эмбеддинги в модель

In [26]:
from gensim.models import FastText

In [27]:
texts = all_tweets_data.text.apply(preprocess).to_list()

In [28]:
model = FastText(texts, size=100, window=5, min_count=1)

In [29]:
weights = np.zeros((len(word2id), 100))
count = 0
for word, i in word2id.items():
    if word == 'PAD':
        continue   
    try:
        weights[i] = model.wv[word]    
    except KeyError:
      count += 1
      # oov словам сопоставляем случайный вектор
      weights[i] = np.random.normal(0,0.1,100)

In [30]:
fasttext_model = CNN(len(word2id), 100, pretrained=True)
train_model(fasttext_model, 0.001, 3)


starting Epoch 0
Training...
F-score: 0.6879616975784302, loss: 0.5653806128480034

Validating...
F-score: 0.74798583984375, loss: 0.5266508212462803

starting Epoch 1
Training...
F-score: 0.7758302688598633, loss: 0.45974038883896573

Validating...
F-score: 0.7539206743240356, loss: 0.4945674329281526

starting Epoch 2
Training...
F-score: 0.824968159198761, loss: 0.38293773979368934

Validating...
F-score: 0.7707210779190063, loss: 0.5062397499851179


FastText помог улучшить качество! Вероятно, модели не хватило эпох, чтобы обучить эмбеддинги, но она лучше работает с предобученными

### Меняем гиперпараметры

Увеличим размер эмбеддинга:

In [31]:
model = CNN(len(word2id), 50)
train_model(model, 0.001, 3)


starting Epoch 0
Training...
F-score: 0.6716139912605286, loss: 0.5812360862603645

Validating...
F-score: 0.7122611403465271, loss: 0.5352640655548515

starting Epoch 1
Training...
F-score: 0.7596361637115479, loss: 0.48372277561759075

Validating...
F-score: 0.7534310817718506, loss: 0.5062956153001368

starting Epoch 2
Training...
F-score: 0.8064237833023071, loss: 0.41261900751227554

Validating...
F-score: 0.7586255073547363, loss: 0.5044789119651186


В целом увеличение эмбеддинга дало лучший результат, так как можно хранить больше информации. Это улучшает качество классификации!

Теперь уменьшим лернинг рейт:

In [32]:
model = CNN(len(word2id), 8)
train_model(model, 0.0005, 3)


starting Epoch 0
Training...
F-score: 0.6075225472450256, loss: 0.643439017751153

Validating...
F-score: 0.6385930776596069, loss: 0.6080097086170665

starting Epoch 1
Training...
F-score: 0.67527836561203, loss: 0.583494718482111

Validating...
F-score: 0.690639853477478, loss: 0.5764812672003704

starting Epoch 2
Training...
F-score: 0.7140663862228394, loss: 0.5424197400553606

Validating...
F-score: 0.6893985867500305, loss: 0.557993971284253


Уменьшение лернинг рейта испортила качество модели, так как, вероятно, модель не дошла до локального минимума, двигаясь со слишком маленьким шагом

### 3. Изменим предобработку

Теперь попробуем прогнать нашу модель на лемматизированных текстах:

In [33]:
train_dataset = TweetsDataset(train_sentences, lemma2id, DEVICE, text='lemmas')
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=64)
val_dataset = TweetsDataset(val_sentences, lemma2id, DEVICE, text='lemmas')
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=64)

In [34]:
model = CNN(len(lemma2id), 10)
train_model(model, 0.001, 3)


starting Epoch 0
Training...
F-score: 0.6683898568153381, loss: 0.5769473152348118

Validating...
F-score: 0.7003044486045837, loss: 0.5321198603077903

starting Epoch 1
Training...
F-score: 0.7483500242233276, loss: 0.49594231518392673

Validating...
F-score: 0.7401742935180664, loss: 0.49363468753434037

starting Epoch 2
Training...
F-score: 0.7804466485977173, loss: 0.4506233713900925

Validating...
F-score: 0.7715306282043457, loss: 0.4832143023589098


С леммами повышается качество классификации, что ожидаемо, так как теперь у него больше данных про конкретное слово, потому что токены объединились в меньшее количество лемм

# Часть II.

Зададим датасет с двумя выходами:

In [35]:
class TwoWayTweetsDataset(Dataset):

    def __init__(self, dataset, word2id, symbol2id, DEVICE):
        self.dataset = dataset['text'].values
        self.word2id = word2id
        self.symbol2id = symbol2id
        self.length = dataset.shape[0]
        self.target = torch.Tensor(dataset['tone'].values)
        self.device = DEVICE

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        tokens = self.tokenize(self.dataset[index]) # токенизируем
        symbols = list(self.dataset[index])
        ids = torch.LongTensor([self.word2id[token] for token in tokens if token in self.word2id])
        symbol_ids = torch.LongTensor([self.symbol2id[symbol] for symbol in symbols if symbol in self.symbol2id])
        y = self.target[index]
        return ids, symbol_ids, y
    
    def preprocess(self, text):
        tokens = text.lower().split()
        tokens = [token.strip(punctuation) for token in tokens]
        tokens = [token for token in tokens if token not in punctuation]
        return tokens

    def tokenize(self, text):
        tokens = [token for token in tokenizer.tokenize(text.lower()) if token not in punctuation]
        #lemmas = [morph.parse(token)[0].normal_form for token in tokens if token not in punctuation]
        return tokens

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, symbol_ids, y = list(zip(*batch))
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      padded_symbol_ids = pad_sequence(symbol_ids, batch_first=True).to(self.device)
      y = torch.Tensor(y).to(self.device)
      return padded_ids, padded_symbol_ids, y

In [36]:
class MixedModel(nn.Module):
    
    def __init__(self, word_vocab_size, symbol_vocab_size, embedding_dim):
        super().__init__()
        self.word_embedding = nn.Embedding(word_vocab_size, embedding_dim)
        self.symbol_embedding = nn.Embedding(symbol_vocab_size, embedding_dim)
        self.linear = nn.Linear(in_features=embedding_dim, out_features=10)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=10, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=8, kernel_size=3, padding='same')
        self.pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=28, out_features=1)
        self.dropout = nn.Dropout(p=0.5)
        self.out = nn.Sigmoid()

    def forward(self, word, symbol):
        word_embedded = self.word_embedding(word)
        sentence = torch.mean(word_embedded, axis=1)
        linear = self.linear(sentence)
        symbol_embedded = self.symbol_embedding(symbol)
        embedded = symbol_embedded.transpose(1,2)
        feature_map_bigrams = self.pooling(self.relu(self.bigrams(embedded)))
        feature_map_trigrams = self.pooling(self.relu(self.trigrams(embedded)))
        pooling1 = feature_map_bigrams.max(2)[0] 
        pooling2 = feature_map_trigrams.max(2)[0]
        concat = torch.cat((pooling1, pooling2, linear), 1)
        logits = self.hidden(concat)
        logits = self.out(logits)      
        return logits

In [37]:
train_dataset = TwoWayTweetsDataset(train_sentences, lemma2id, symbol2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=512)
val_dataset = TwoWayTweetsDataset(val_sentences, lemma2id, symbol2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=512)

In [38]:
def train(model, iterator, optimizer, criterion, metric):
    print('Training...')
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах
    epoch_metric = 0
    model.train()

    for i, (texts, symbols, ys) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(texts, symbols).squeeze()  #прогоняем данные через модель
        loss = criterion(preds, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        batch_metric = metric(preds.round().long(), ys.long())
        epoch_metric += batch_metric
        
    print(f'F-score: {epoch_metric / len(iterator)}, loss: {epoch_loss / len(iterator)}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator)# возвращаем среднее значение по всей выборке

In [39]:
def evaluate(model, iterator, criterion, metric):
    print("\nValidating...")
    epoch_loss = 0
    epoch_metric = 0
    model.eval() 
    with torch.no_grad():
        for i, (texts, symbols, ys) in enumerate(iterator):   
            preds = model(texts, symbols).squeeze()  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            batch_metric = metric(preds.round().long(), ys.long())
            epoch_metric += batch_metric

    print(f'F-score: {epoch_metric / len(iterator)}, loss: {epoch_loss / len(iterator)}')
        
    return epoch_metric / len(iterator), epoch_loss / len(iterator)# возвращаем среднее значение по всей выборке

В целом качество этой модели лучше, чем предыдущей, даже с меньшим лернинг рейтом

In [42]:
model = MixedModel(len(lemma2id), len(symbol2id), 8)
train_model(model, 0.0001, 3)


starting Epoch 0
Training...
F-score: 0.5283611416816711, loss: 0.6911715675407732

Validating...
F-score: 0.5676530003547668, loss: 0.6839378509628639

starting Epoch 1
Training...
F-score: 0.6234907507896423, loss: 0.6717464334528211

Validating...
F-score: 0.7041453123092651, loss: 0.6528091651670048

starting Epoch 2
Training...
F-score: 0.7885995507240295, loss: 0.6144483237199381

Validating...
F-score: 0.8563372492790222, loss: 0.5650047077221817


In [43]:
model = MixedModel(len(lemma2id), len(symbol2id), 8)
train_model(model, 0.001, 3)


starting Epoch 0
Training...
F-score: 0.863972544670105, loss: 0.37670726998591086

Validating...
F-score: 0.9870110750198364, loss: 0.07366102942255105

starting Epoch 1
Training...
F-score: 0.9970766305923462, loss: 0.025617957445727268

Validating...
F-score: 0.9994733333587646, loss: 0.008515787420773439

starting Epoch 2
Training...
F-score: 0.9994387626647949, loss: 0.005774067828274319

Validating...
F-score: 0.9996927976608276, loss: 0.0035480288186979093


In [44]:
train_dataset = TwoWayTweetsDataset(train_sentences, word2id, symbol2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=512)
val_dataset = TwoWayTweetsDataset(val_sentences, word2id, symbol2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=512)

In [45]:
model = MixedModel(len(word2id), len(symbol2id), 8)
train_model(model, 0.001, 3)


starting Epoch 0
Training...
F-score: 0.905828058719635, loss: 0.31059286847290857

Validating...
F-score: 0.9785996675491333, loss: 0.062060048615329724

starting Epoch 1
Training...
F-score: 0.9871753454208374, loss: 0.037329780223818734

Validating...
F-score: 0.9963210225105286, loss: 0.017604546297048586

starting Epoch 2
Training...
F-score: 0.998002827167511, loss: 0.011016009926376209

Validating...
F-score: 0.9989983439445496, loss: 0.006635223790019584


При увеличении лернинг рейта увеличивается качество! При этом качество хорошее и на леммах, и на токенах

## Анализ

1. Модель с символьными и пословными эмбеддингами работает лучше: вероятно, это можно объяснить, что не вся пунктуация убирается, и некоторые символьные значения помогают однозначно определять тональность

2. Для обеих моделей оптимален learning rate 0.001, при таком лернинг рейте они достигают минимума (или приближаются к нему), при меньшем lr шаг слишком маленький.

3. Предобученные эмбеддинги работают лучше, чем обучаемые внутри модели. Вероятно, модель не успевает за три эпохи хорошо обучить эмбеддинги, поэтому ей помогают предобученные эмбеддинги.

4. Увеличение размера эмбеддингов увеличивает качество, так как хранится больше информации.

5. С леммами модель работает лучше, так как объединяет токены в одну лемму и получает о ней больше информации